In [1]:
# Ejecutando esta celda permito que las funciones
# que importe en este notebook se actualizen
# automaticamente si las cambio en los scripts
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp2d


# Con este comando podre importar las funciones
# desde los scripts (asi trabajamos directamente
# en los scripts y evitamos tener una copia de cada
# funcion en el notebook)
import sys
sys.path.insert(0, '..')

from src.velocity import interpolate_discrete
from src.velocity import velocity_composition
from src.velocity import boat_movement
from src.velocity import desired_velocity

## Definitions

+ I call `boat_velocity` to the equivalent velocity of the boat if there were not streams.
+ I call `stream_velocity` to the velocity of the stream. 
+ I call `velocity` to the composition between both velocities

Velocities are `arrays` expressed as 
```velocity = np.array([vel_latitude, vel_longitude])```

In [26]:
lat = 0
long = 3

final_point = np.array([0.1234, 3.2221])

vel_array = np.random.RandomState(seed=2).rand(10,20,2)
array_lat = np.arange(-5,5)
array_long = np.arange(-13,7)
ts = 0.1

In [27]:
boat_velocity = desired_velocity(np.array([lat, long]),
                 final_point,
                 vel_array, array_lat, array_long, ts)

In [28]:
final_point_estimated = np.array(boat_movement(boat_velocity,
                 lat, long, vel_array, array_lat, array_long, ts))

In [29]:
print(f'The original boat velocity is: {boat_velocity}')
print(f'The original final point is: {final_point},\nthe estimated final point is {final_point_estimated}')

The original boat velocity is: [0.90325297 1.99238975]
The original final point is: [0.1234 3.2221],
the estimated final point is [0.1234 3.2221]
